In [1]:
#### Entity Matching Pipeline with rule based matching ####
###########################################################

#### Notebook showing the implementation of an EM pipeline with rule-based matching.
# Attribute based blocking is done, along with three matching rules on the basis of 
# features obtained from the similarity values of the attributes

# Dataset exploration #
# After going through multiple datasets like movies.csv; citations.csv etc., we chose the music dataset. 
# In other datasets, we had issues like inaccessibility, same attributes not being present across the two datasets etc. 
# The two music datasets have similar attributes across them. Attributes like Genre etc. can also be used as sensitive attributes.
# The labeled dataset of size 539 was also available for music datasets for evaluation purpose.

In [2]:
# Importing the required libraries
import sys
import py_entitymatching as em
import pandas as pd
import os

In [3]:
# Load csv files as dataframes and set the key attribute in the dataframe
path_A = 'music1.csv'
path_B = 'music2.csv'
A = em.read_csv_metadata(path_A, key='Sno')
B = em.read_csv_metadata(path_B, key='Sno')

Metadata file is not present in the given path; proceeding to read the csv file.
Metadata file is not present in the given path; proceeding to read the csv file.


In [4]:
# Display number of tuples in the datasets
print('Number of tuples in A: ' + str(len(A)))
print('Number of tuples in B: ' + str(len(B)))
print('Number of tuples in A X B (i.e the cartesian product): ' + str(len(A)*len(B)))

Number of tuples in A: 6907
Number of tuples in B: 55923
Number of tuples in A X B (i.e the cartesian product): 386260161


In [5]:
# Displaying first two entries from the first music dataset
A.head(2)

,Sno,Album_Name,Album_Price,Artist_Name,CopyRight,Customer_Rating,Genre,Price,Released,Song_Name,Time
0,1,Welcome to Cam Country - EP,$4.26,Cam,2015 Sony Music Entertainment,4.72396,"Country,Music,Contemporary Country,Honky Tonk",$0.99,31-Mar-15,Runaway Train,3:01
1,2,Me 4 U,$9.99,Omi,"2015 Ultra Records, LLC under exclusive license to Columbia Records, a Division of Sony Music E...",3.38158,"Pop/Rock,Music,Pop,Dance,R&B/Soul",Album Only,NaN,Track 14,3:41


In [6]:
# Displaying first two entries from the second music dataset
B.head(2)

,Sno,Album_Name,Artist_Name,Song_Name,Price,Time,Released,Label,Copyright,Genre
0,1,! (Volume 2) [Explicit],Rusko,Saxophone Stomp [Explicit],$1.29,3:20,"September 16, 2014",Decca International,(C) 2014 FMLY Under Exclusive License To Universal Music Canada Inc.,"Dance & Electronic,Dubstep"
1,2,! (Volume 2) [Explicit],Rusko,I Wanna Mingle [feat. Pusher],$1.29,2:36,"September 16, 2014",Decca International,(C) 2014 FMLY Under Exclusive License To Universal Music Canada Inc.,"Dance & Electronic,Dubstep"


In [7]:
# Display the keys of the input tables
print(em.get_key(A), em.get_key(B))
# If the tables are large we can downsample the tables like this
A1, B1 = em.down_sample(A, B, 1000, 1, show_progress=False)
print("Lengths after downsampling-")
print(len(A1), len(B1))

Sno Sno
Lengths after downsampling-
564 1000


Blocking

In [8]:
# Create attribute equivalence blocker
ab = em.AttrEquivalenceBlocker()

# Block using artist_name attribute
C = ab.block_tables(A1, B1, "Artist_Name","Artist_Name",
                    l_output_attrs=["Sno", "Album_Name", "Artist_Name", "CopyRight", "Released", "Song_Name" ,"Time"], 
                    r_output_attrs=["Sno", "Album_Name", "Artist_Name", "Copyright", "Released", "Song_Name", "Time"]
                    )

In [9]:
# Printing length of candidate set
len(C)

1288

Matching

In [10]:
# Sample candidate set
# S = em.sample_table(C, 538)

In [11]:
# Label S
# G = em.label_table(S, 'label')

In [12]:
# Load the pre-labeled data
path_G = 'music_labeled_data.csv'
G = em.read_csv_metadata(path_G, 
                         key='_id',
                         ltable=A, rtable=B, 
                         fk_ltable='ltable.Sno', fk_rtable='rtable.Sno')
print(len(G))

Metadata file is not present in the given path; proceeding to read the csv file.


539


In [13]:
# Initializing the rule based matcher
brm = em.BooleanRuleMatcher()

In [14]:
# Generate features
feature_table = em.get_features_for_matching(A, B, validate_inferred_attr_types=False)

In [15]:
# Listing the names of the features generated
feature_table['feature_name']

0                                     Sno_Sno_exm
1                                     Sno_Sno_anm
2                                Sno_Sno_lev_dist
3                                 Sno_Sno_lev_sim
4           Album_Name_Album_Name_jac_qgm_3_qgm_3
5       Album_Name_Album_Name_cos_dlm_dc0_dlm_dc0
6       Album_Name_Album_Name_jac_dlm_dc0_dlm_dc0
7                       Album_Name_Album_Name_mel
8                  Album_Name_Album_Name_lev_dist
9                   Album_Name_Album_Name_lev_sim
10                      Album_Name_Album_Name_nmw
11                       Album_Name_Album_Name_sw
12        Artist_Name_Artist_Name_jac_qgm_3_qgm_3
13    Artist_Name_Artist_Name_cos_dlm_dc0_dlm_dc0
14    Artist_Name_Artist_Name_jac_dlm_dc0_dlm_dc0
15                    Artist_Name_Artist_Name_mel
16               Artist_Name_Artist_Name_lev_dist
17                Artist_Name_Artist_Name_lev_sim
18                    Artist_Name_Artist_Name_nmw
19                     Artist_Name_Artist_Name_sw


In [16]:
# Since feature related to time is not present due to the difference in data types, we create the time feature manually.
# Creating function for checking if the two timevalues are equal or not.
def time_feature(ltuple, rtuple):
    p1 = ltuple.Time
    p2 = rtuple.Time
    if p1 == p2:
        return 1.0
    else:
        return 0.0

# Add feature to the feature table
em.add_blackbox_feature(feature_table, 'time_time_feature', time_feature)

True

In [17]:
# Add two rules to the rule-based matcher

# The first rule compares the album names
brm.add_rule(['Album_Name_Album_Name_lev_sim(ltuple, rtuple) > 0.8'], feature_table)
# This second rule compares the genres
brm.add_rule(['Genre_Genre_lev_sim(ltuple, rtuple) > 0.8'], feature_table)
# This third rule compares the times
brm.add_rule(['time_time_feature(ltuple, rtuple) == 1'],feature_table)
brm.get_rule_names()

odict_keys(['_rule_0', '_rule_1', '_rule_2'])

In [18]:
brm.predict(G, target_attr='pred_label', append=True)
G.head(5)

,Unnamed: 0,_id,ltable.Sno,rtable.Sno,ltable.Album_Name,ltable.Artist_Name,ltable.CopyRight,ltable.Released,ltable.Song_Name,ltable.Time,rtable.Album_Name,rtable.Artist_Name,rtable.CopyRight,rtable.Released,rtable.Song_Name,rtable.Time,label,pred_label
0,916,916,111,53124,vhs,x ambassadors,2015 kidinakorner/interscope records,30-Jun-15,vhs outro (interlude),1:25,vhs [explicit],x ambassadors,(c) 2015 kidinakorner/interscope records,"June 30, 2015",vhs outro (interlude) [explicit],1:25,1,1
1,1053,1053,148,50767,title (deluxe),meghan trainor,"2014, 2015 epic records, a division of sony music entertainment",9-Jan-15,credit,2:51,title (deluxe),meghan trainor,"2011 what a music ltd, licence exclusive parlophone music france","January 9, 2015",credit,2:51,1,1
2,1290,1290,206,41214,slow down (remixes),selena gomez,"2013 hollywood records, inc.",20-Aug-13,slow down (smash mode remix),5:21,slow down remixes,selena gomez,"(c) 2013 hollywood records, inc.","August 20, 2013",slow down (smash mode remix),5:21,1,1
3,1424,1424,211,19812,slow down (reggae remixes) - single,selena gomez,"2013 hollywood records, inc.",20-Aug-13,slow down (sure shot rockers reggae dub remix),3:15,good for you (remixes),selena gomez,(c) 2015 interscope records,"September 4, 2015",good for you (yellow claw & cesqeaux remix) [feat. a$ap rocky],3:01,0,0
4,1706,1706,250,53111,vhs,x ambassadors,2015 kidinakorner/interscope records,30-Jun-15,vhs outro (interlude),1:25,vhs [explicit],x ambassadors,(c) 2015 kidinakorner/interscope records,"June 30, 2015",first show (interlude),0:11,0,0


In [19]:
# Evaluate the predictions
eval_result = em.eval_matches(G, 'label', 'pred_label')
em.print_eval_summary(eval_result)

Precision : 71.95% (118/164)
Recall : 89.39% (118/132)
F1 : 79.73%
False positives : 46 (out of 164 positive predictions)
False negatives : 14 (out of 375 negative predictions)


In [20]:
########## For involving fairness constraints, we take Genre as the sensitive attributes
#### The following distribution of distinct categories was obtained for the two datasets-->
# Music 1 : {'Pop': 1758, 'Hip-Hop': 685, 'Rock': 515, 'Dance': 952, 'Country': 1367, 
# 'Electronic': 211, 'Alternative': 484, 'Soundtrack': 397, 'Other': 538}
# Music 2 : {'Pop': 7130, 'Rap & Hip-Hop': 0, 'Rock': 3248, 'Dance': 8274, 'Country': 8317, 'Electronic': 4019,
# 'Alternative': 4806, 'Soundtrack': 3594, 'Other': 16535}

#### There were mamy distinct categories in the dataset, they have been clubbed into umbrella categories as shown above.
